In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('resources/test.csv')
X = pd.get_dummies(df.drop('goals', axis=1), drop_first=True)
X = X.dropna()

y = df['goals']
y = y[X.index]


# Remove outliers from y using IQR method
Q1 = y.quantile(0.25)
Q3 = y.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
non_outliers = (y >= lower_bound) & (y <= upper_bound)
X = X[non_outliers]
y = y[non_outliers]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Define hyperparameter grid for Ridge
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
}

# Create Ridge model
ridge_model = Ridge()

# Set up grid search with cross-validation
grid_search_ridge = GridSearchCV(ridge_model, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search_ridge.fit(X_train, y_train)

# Get the best model and predictions
best_ridge_params = grid_search_ridge.best_params_
best_ridge_model = grid_search_ridge.best_estimator_
ridge_predictions = best_ridge_model.predict(X_test)

# Calculate performance metrics
ridge_mse = mean_squared_error(y_test, ridge_predictions)
ridge_r2 = r2_score(y_test, ridge_predictions)
ridge_mae = mean_absolute_error(y_test, ridge_predictions)

# Find best and worst predictions
ridge_errors = abs(ridge_predictions - y_test.values)
best_ridge_idx = ridge_errors.argmin()
worst_ridge_idx = ridge_errors.argmax()

best_ridge_prediction = ridge_predictions[best_ridge_idx]
worst_ridge_prediction = ridge_predictions[worst_ridge_idx]
best_ridge_actual = y_test.iloc[best_ridge_idx]
worst_ridge_actual = y_test.iloc[worst_ridge_idx]

# Print results for Ridge
print(f"Ridge Regression Performance (After Hyperparameter Tuning):")
print(f"Best Alpha: {best_ridge_params['alpha']}")
print(f"R²: {ridge_r2:.2f}")
print(f"MSE: {ridge_mse:.2f}")
print(f"MAE: {ridge_mae:.2f}")
print(f"Best Ridge Prediction: {best_ridge_prediction:.2f}, Actual: {best_ridge_actual:.2f}")
print(f"Worst Ridge Prediction: {worst_ridge_prediction:.2f}, Actual: {worst_ridge_actual:.2f}")

Ridge Regression Performance (After Hyperparameter Tuning):
Best Alpha: 100
R²: 0.56
MSE: 0.01
MAE: 0.07
Best Ridge Prediction: 0.03, Actual: 0.03
Worst Ridge Prediction: 0.04, Actual: 0.40


C:\Users\pascenby\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=3.17868e-18): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
